# 아홉째마당. 지도 시각화

## 서울시 동별 외국인 인구 단계 구분도 만들기


**단계 구분도(choropleth map)**: 지역별 통계치를 색상 차이로 표현한 지도

#### 1. 서울시 동 경계 지도 데이터 준비하기


`GeoJSON`은 위치 정보를 `JSON` 포맷으로 저장한 표준 지리 정보 데이터 포맷으로 지리 정보를 다루는 대부분의 소프트웨어에서 지원.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/data-analytics-101/data/'

In [ ]:
import json
geo_seoul = json.load(open(path + 'EMD_Seoul.geojson', encoding = 'UTF-8'))

In [ ]:
type(geo_seoul)

In [ ]:
# 행정 구역 코드 출력 
geo_seoul['features'][0]['properties']

In [ ]:
# 위도, 경도 좌표 출력
geo_seoul['features'][0]['geometry']

#### 2. 서울시 동별 외국인 인구 데이터 준비하기

In [ ]:
import pandas as pd

foreigner = pd.read_csv(path + 'Foreigner_EMD_Seoul.csv')
foreigner.head()

In [ ]:
foreigner.info()

`foreigner`의 `code` 변수는 `int64` 타입으로 되어 있기 때문에, 지도를 만드는데 사용하기 위해서는 문자열 타입으로 변경해야함.

In [ ]:
foreigner['code'] = foreigner['code'].astype(str)

In [ ]:
foreigner.info()

#### 3. 단계 구분도 만들기

`folium` 패키지를 이용하면 단계 구분도를 그릴 수 있음

In [ ]:
!pip install folium

(1) 배경 지도 만들기

`folium.Map()` 함수

- location : 지도 중심 좌표
- zoom_start : 화면 확대 단계
- tiles : 지도 종류

In [ ]:
import folium

map_seoul = folium.Map(location = [37.56, 127],      
                       zoom_start = 12,           
                       tiles = 'cartodbpositron')
map_seoul

(2) 단계 구분도 만들기

`folium.Choropleth()` 함수

- geo_data : 지도 데이터
- data : 색으로 표현할 통계 데이터
- colums : 통계 데이터의 행정 구역 코드 변수, 색으로 표현할 변수 
- key_on : 지도 데이터의 행정 구역 코드

In [ ]:
folium.Choropleth(geo_data = geo_seoul,
                  data = foreigner,
                  columns = ('code', 'pop'),
                  key_on = 'feature.properties.ADM_DR_CD') \
      .add_to(map_seoul)
map_seoul

지역을 단계별로 나눌 때 기준으로 삼는 계급 구간이 적당하지 않기 때문에 지역에 색상 표현이 잘 되지 않음. `quantile()`를 이용해 계급 구간마다 같은 수의 데이터가 배정되도록 계급 구간(`bins`)을 나눔.

In [ ]:
bins = list(foreigner['pop'].quantile([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))
bins

In [ ]:
map_seoul = folium.Map(location = [37.56, 127],   
                       zoom_start= 12,           
                       tiles = 'cartodbpositron') 

`folium.Choropleth()` 함수

- geo_data : 지도 데이터
- data : 색으로 표현할 통계 데이터
- colums : 통계 데이터의 행정 구역 코드 변수, 색으로 표현할 변수 
- key_on : 지도 데이터의 행정 구역 코드
- fill_color : 컬러맵
- fill_opacity : 투명도
- line_opacity : 경계선 투명도
- bins : 계급 구간 기준값

In [ ]:
folium.Choropleth(geo_data = geo_seoul,
                  data = foreigner,
                  columns = ('code', 'pop'),
                  key_on = 'feature.properties.ADM_DR_CD',
                  fill_color = 'Blues',
                  nan_fill_color = 'White',
                  fill_opacity = 1,
                  line_opacity = 0.5,
                  bins = bins) \
      .add_to(map_seoul)
map_seoul